In [ ]:
import healpy as hp
import mapsims
from glob import glob
import numpy as np
from collections import defaultdict

In [ ]:
%matplotlib inline

In [ ]:
import pickle

In [ ]:
sim = mapsims.from_config(["common.toml", "noise.toml"], override={"channels":"tube:ST0"})

noise = sim.other_components["noise"]

In [ ]:
hitmaps = {}
ave_hits = {}
sky_fractions = defaultdict(dict)

for i_tube in range(4):
    tube = f"ST{i_tube}"
    chs = mapsims.parse_channels("tube:"+tube)[0]
    hitmaps[tube], ave_hits[tube] = noise.get_hitmaps(tube)
    for i in [0,1]:
        mask = hitmaps[tube][i] > 0
        sky_fractions[tube][i] = (noise.pixarea_map*mask).sum()/np.pi/4
    for ch, h in zip(chs, hitmaps[tube]):
        h /= h.max()

In [ ]:
sky_fractions

In [ ]:
cl = {}

for i_tube in range(4):
    tube = f"ST{i_tube}"
    cl[tube] = pickle.load(open(f"output/noise/N_ell_tube_{tube}.pkl", "rb")) 

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import mapsims

In [ ]:
from glob import glob

In [ ]:
import matplotlib
matplotlib.rc('text', usetex=False)
fontProperties = {'family':'sans-serif',
                  'weight' : 'normal', 'size' : 16}

In [ ]:
import pickle

In [ ]:
%matplotlib inline

In [ ]:
fig, ax = plt.subplots(nrows=4, ncols=2,sharey=True, figsize=(20/3*2, 5*4))
pol = 0
i_tube = 0
for (tube, chs) in noise.tubes.items():
    ell, ps_T, ps_P = noise.get_fullsky_noise_spectra(tube=tube)
    if tube.startswith("S"):
        wn = noise.get_white_noise_power(tube, sky_fractions[tube][i])

        m = {}
        for i, ch in enumerate(chs):
            len_cl = len(cl[tube][i])
            ax[i_tube][i].loglog(ell, ps_T[i] * sky_fractions[tube][i], label="Expected T spectrum")
            ax[i_tube][i].loglog(cl[tube][i][pol], label="I", alpha=.5)
            ax[i_tube][i].hlines(wn[i], 100, ell[-1], label="White noise power")

            #ax[i_tube][i].set_ylim(1e-8,1e-2)
            ax[i_tube][i].set_title(ch, fontsize=16)
            
            ax[i_tube][i].set_ylabel(r"N_ell [muK^2]", fontsize=16)
            ax[i_tube][i].legend()
            ax[i_tube][i].grid(True)
        i_tube += 1
        
plt.savefig("plots/SAT_T.png")

In [ ]:
cl[tube][i][pol][-100:].mean() / (2*wn[i])

In [ ]:
wn[i]

In [ ]:
fig, ax = plt.subplots(nrows=4, ncols=2, sharey=True, figsize=(20/3*2, 5*4))
ylim = (1e-7, 1e-2)
i_tube = 0
pol_label = "TQU"
for (tube, chs) in noise.tubes.items():
    ell, ps_T, ps_P = noise.get_fullsky_noise_spectra(tube=tube)
    if tube.startswith("S"):

        #nside = hp.npix2nside(len(hitmaps[tube][0]))
        #pixarea = hp.nside2pixarea(nside)
        wn = noise.get_white_noise_power(tube, sky_fractions[tube][i])
        m = {}
        for i, ch in enumerate(chs):
            ax[i_tube][i].loglog(ell, ps_P[i] * sky_fractions[tube][i], label="Expected P spectrum")
            # noise power is double in polarization
            ax[i_tube][i].hlines(wn[i] * 2, 100, ell[-1], label="White noise power")
            ax[i_tube][i].set_ylabel(r"N_ell [muK^2]", fontsize=16)
        for pol in [1,2]:
            for i, ch in enumerate(chs):
                len_cl = len(cl[tube][i])
                ax[i_tube][i].loglog(cl[tube][i][pol], label=pol_label[pol], alpha=.5)

                #ax[i_tube][i].set_ylim(*ylim)
                ax[i_tube][i].set_title(ch, fontsize=16)
                ax[i_tube][i].legend()
                ax[i_tube][i].grid(True)
        i_tube += 1
    
    

plt.savefig("plots/SAT_P.png")